In [1]:
# pip install awswrangler

In [2]:
# pip install spacy && python -m spacy download pt_core_news_sm

In [3]:
# !pip install sagemaker-studio-image-build

In [4]:
import os
import sys
import sagemaker
from sklearn.pipeline import Pipeline

sys.path.append(os.path.dirname(os.getcwd())+'/src')
from train import *
from preprocess import preprocess_module 
from feature_engineering import feature_engineering_pipeline

In [5]:
stage = 'dev'
bucket = f'ons-ds-{stage}-entityrecognition'

print(f'Initialized training and deployment sequence at {datetime.now()}')

Initialized training and deployment sequence at 2022-09-06 14:24:47.790316


---

In [6]:
#-------------------------------------------------------------------------
#training model for predefined entities   
predefined_model_id, predefined_metrics = run_training_pipeline(
    bucket=bucket,
    model='predefined',
    train_from_blank=True
    )

predefined_metrics = load_metrics_from_s3(
    bucket=bucket,
    path='mpo/mvp/artifacts/model_info/predefined/metrics.parquet'
    )

predefined_production_variant = get_production_variant(
    predefined_metrics
    )

predefined_status = check_production_variant_availability(
    production_variant=predefined_production_variant
    )

production variant selected: 2022-09-05-22-35-21


In [7]:
#-------------------------------------------------------------------------
#training model for inequalities entities
inequalities_model_id, inequalities_metrics = run_training_pipeline(
    bucket=bucket,
    model='inequalities',
    train_from_blank=False
    )

inequalities_metrics = load_metrics_from_s3(
    bucket=bucket,
    path='mpo/mvp/artifacts/model_info/inequalities/metrics.parquet'
    )

inequalities_production_variant = get_production_variant(
    inequalities_metrics
    )

inequalities_status = check_production_variant_availability(
    production_variant=inequalities_production_variant
    )

production variant selected: 2022-09-05-22-38-29


In [8]:
#-------------------------------------------------------------------------
# deployment of new variant(s)
if (predefined_status == False) | (inequalities_status==False):

    prepare_production_variant(
        production_variant=predefined_production_variant,
        model='predefined',
        bucket=bucket
        )

    prepare_production_variant(
        production_variant=inequalities_production_variant,
        model='inequalities',
        bucket=bucket
        )

    deployment_sequence(
        predefined_id=predefined_production_variant,
        inequalities_id=inequalities_production_variant,
        bucket=bucket,
        stage=stage
        )

mpo/mvp/artifacts/model/predefined/2022-09-05-22-35-21.zip
loaded 2022-09-05-22-35-21
mpo/mvp/artifacts/model/inequalities/2022-09-05-22-38-29.zip
loaded 2022-09-05-22-38-29
uploaded image to ECR
Model name: mpo-ner-dev-2022-09-05-22-35-21-2022-09-05-22-38-29
Container image: 478704051461.dkr.ecr.us-east-1.amazonaws.com/ons-ds-entity-recognition:latest
Model Arn: arn:aws:sagemaker:us-east-1:478704051461:model/mpo-ner-dev-2022-09-05-22-35-21-2022-09-05-22-38-29
uploaded sagemaker model: mpo-ner-dev-2022-09-05-22-35-21-2022-09-05-22-38-29
Endpoint config name: mpo-ner-dev-endpoint-config
Endpoint config Arn: arn:aws:sagemaker:us-east-1:478704051461:endpoint-config/mpo-ner-dev-endpoint-config
uploaded sagemaker endpoint configuration: mpo-ner-dev-endpoint-config
Endpoint name: mpo-ner-dev-endpoint
Endpoint Arn: arn:aws:sagemaker:us-east-1:478704051461:endpoint/mpo-ner-dev-endpoint
Endpoint Status: Creating
Waiting for mpo-ner-dev-endpoint endpoint to be in service...
deployed endpoint: mp

---

In [11]:
import json
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

content_type = "application/json"
request_body = {"fragmento": "A UHE Itaipu 60 hz deve ter a sua geração limitada. Tensão da SE que inicia a manobra inferior ou igual a 242 kV"}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName='mpo-ner-dev-endpoint',
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['entidades']
result

[{'tipo_entidade': 'USINA', 'entidade': 'UHE Itaipu'},
 {'tipo_entidade': 'VALOR_COM_UNID.MEDIDA', 'entidade': '60 hz'},
 {'tipo_entidade': 'SUBESTACAO', 'entidade': 'SE que inicia a'},
 {'tipo_entidade': 'OPERADOR_MATEMATICO', 'entidade': 'inferior ou igual a'},
 {'tipo_entidade': 'VALOR_COM_UNID.MEDIDA', 'entidade': '242 kV'},
 {'tipo_entidade': 'INEQUACAO',
  'entidade': 'Tensão da SE que inicia a manobra inferior ou igual a 242 kV'}]

---

In [10]:
delete_endpoint(endpoint_name)

In [58]:
delete_endpoint_configuration(endpoint_config_name)

Configuração de endpoint ons-ds-dev-entity-recognition-endpoint-config excluída com sucesso


In [60]:
delete_model('ons-mvp-entity-recognition-model-2022-09-01-13-51-35')

Modelo ons-mvp-entity-recognition-model-2022-09-01-13-51-35 excluído com sucesso
